# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Wallet.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Wallet

This notebook holds all the code around handling a wallet of private and public keys.

**Please be careful with your private keys, and don't provide them to any code if you are not 100% confident you know what it will do.**

In [ ]:
import json
import logging
import os.path
import typing

from solana.account import Account
from solana.publickey import PublicKey
from spl.token.client import Token
from spl.token.constants import TOKEN_PROGRAM_ID

from Context import AccountInfo, Context


## Wallet class

The `Wallet` class wraps our understanding of saving and loading keys, and creating the appropriate Solana `Account` object.

To load a private key from a file, the file must be a JSON-formatted text file with a root array of the 64 bytes making up the secret key.

For example:
```
[200,48,184,13... for another 60 bytes...]
```
**TODO:** It would be good to be able to load a `Wallet` from a mnemonic string. I haven't yet found a Python library that can generate a BIP44 derived seed for Solana that matches the derived seeds created by Sollet and Ledger.

In [ ]:
_DEFAULT_WALLET_FILENAME: str = "id.json"


class Wallet:
    def __init__(self, secret_key):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.secret_key = secret_key[0:32]
        self.account = Account(self.secret_key)

    @property
    def address(self) -> PublicKey:
        return self.account.public_key()

    def fetch_largest_token_account(self, context: Context, token_mint: PublicKey) -> typing.Optional[AccountInfo]:
        largest_account = context.fetch_largest_token_account_for_owner(self.address, token_mint)
        return largest_account

    def fetch_or_create_largest_token_account(self, context: Context, token_mint: PublicKey) -> AccountInfo:
        token_account = self.fetch_largest_token_account(context, token_mint)
        if token_account is not None:
            return token_account
        self.logger.info(f"No token account for mint {token_mint}, so creating it...")
        token = Token(context.client, token_mint, TOKEN_PROGRAM_ID, self.account)
        token.create_account(self.address)
        token_account = context.load_account(token_mint)
        if token_account is None:
            raise Exception(f"Could not create token account for mint {token_mint} from wallet {self.address}.")
        return token_account

    def save(self, filename: str, overwrite: bool = False):
        if os.path.isfile(filename) and not overwrite:
            raise Exception(f"Wallet file '{filename}' already exists.")

        with open(filename, "w") as json_file:
            json.dump(list(self.secret_key), json_file)

    @staticmethod
    def load(filename: str = _DEFAULT_WALLET_FILENAME) -> "Wallet":
        if not os.path.isfile(filename):
            logging.error(f"Wallet file '{filename}' is not present.")
            raise Exception(f"Wallet file '{filename}' is not present.")
        else:
            with open(filename) as json_file:
                data = json.load(json_file)
                return Wallet(data)

    @staticmethod
    def create() -> "Wallet":
        new_account = Account()
        new_secret_key = new_account.secret_key()
        return Wallet(new_secret_key)


# default_wallet object

A default Wallet object that loads the private key from the id.json file, if it exists.

In [ ]:
default_wallet = None
if os.path.isfile(_DEFAULT_WALLET_FILENAME):
    try:
        default_wallet = Wallet.load(_DEFAULT_WALLET_FILENAME)
    except Exception as exception:
        logging.warning(f"Failed to load default wallet from file '{_DEFAULT_WALLET_FILENAME}' - exception: {exception}")


# 🏃 Running

A simple harness to load a `Wallet` if the `wallet.json` file exists, and print out its `PublicKey`.

**Please be careful with your private keys!**

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    import os.path
    from Context import default_context

    filename = _DEFAULT_WALLET_FILENAME
    if not os.path.isfile(filename):
        print(f"Wallet file '{filename}' is not present.")
    else:
        wallet = Wallet.load(filename)
        print("Wallet address:", wallet.address)
        # SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt is Serum's mint address
        print(wallet.fetch_largest_token_account(default_context, PublicKey("SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt")))
